<a href="https://colab.research.google.com/github/JoelRai07/Projekt-Vetter-SQL/blob/main/Fallstudie_Juicy_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Elektro-LKWs

In [14]:
! pip install -q pyscipopt

In [15]:
import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [16]:
! git clone https://github.com/UP24Erz/Kressner_Projekt.git

fatal: destination path 'Kressner_Projekt' already exists and is not an empty directory.


In [17]:
! ls Kressner_Projekt/Daten

chargers.csv  diesel_trucks.csv  electric_trucks.csv  routes.csv


In [18]:
folder = "Kressner_Projekt/Daten"

In [19]:
ladesäulen = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [20]:
ladesäulen.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [21]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [22]:
diesel_trucks.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [23]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [24]:
electric_trucks.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [25]:
routen = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [26]:
routen.head(20)

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00


## Indexmengen

In [27]:
I = kapazitaeten.Produktionsstandort.unique()
J = list(nachfrage.Markt.unique())
A = kapazitaeten.Ausbaustufe.unique()

NameError: name 'kapazitaeten' is not defined

## Erweiterung Aufgabe 2

In [ ]:
J.append("Arabische_Halbinsel")

In [ ]:
J

## Parameter

In [ ]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]

## Erweiterung Aufgabe 2

In [ ]:
cp["Charleston","Arabische_Halbinsel"]=125
cp["Curitiba","Arabische_Halbinsel"]=100
cp["Hamburg","Arabische_Halbinsel"]=100
cp["Quanzhou","Arabische_Halbinsel"]=75
cp["East London","Arabische_Halbinsel"]=85

In [ ]:
# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]

## Erweiterung Aufgabe 1

In [ ]:
cf['Charleston', 'klein'] = cf['Charleston', 'klein'] * 0.7
cf['Charleston', 'groß'] = cf['Charleston', 'groß'] * 0.7

In [ ]:
cf

In [ ]:
# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
cap

In [ ]:
# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
d


## Erweiterung Aufgabe 2

In [ ]:
d["Arabische_Halbinsel"]=5

## Modellinstanz ziehen

In [ ]:
model = Model()

## Entscheidungsvariablen

In [ ]:
Y = {}
for i in I:
  for a in A:
    Y[i,a] = model.addVar(vtype="B", name=f"Ausbaustufe_{a}_Standort_{i}")

In [ ]:
X = {}
for i in I:
  for j in J:
    X[i,j] = model.addVar(vtype="I", name=f"Produktion_Standort_{i}_für_Markt_{j}")

In [ ]:
len(model.getVars())

## Zielfunktion

In [ ]:
model.setObjective(
    quicksum(cp[i,j]*X[i,j] for i in I for j in J)
    + quicksum(cf[i,a]*Y[i,a] for i in I for a in A),
    sense="minimize"
    )

## Nebenbedingungen

In [ ]:
# Nachfrage (5)
for j in J:
  model.addCons(quicksum(X[i,j] for i in I) >= d[j], name=f"Nachfrage_Markt_{j}")

In [ ]:
# Kopplung Produktion und Investment (5)
for i in I:
  model.addCons(
      quicksum(X[i,j] for j in J) <= quicksum(cap[a]*Y[i,a] for a in A),
      name=f"Kapazität_Standort_{i}"
      )

In [ ]:
# Nur eine Ausbaustufe (5)
for i in I:
  model.addCons(
      quicksum(Y[i,a] for a in A) <= 1,
      name=f"Ausbaustufe_{i}"
      )

## Erweiterung Aufgabe 3

In [ ]:
# Mindestbeschäftigung
model.addCons(quicksum(X["Charleston",j] for j in J) >= 5, name="Mindestmenge_Charleston")
model.addCons(quicksum(X["Hamburg",j] for j in J) >= 20, name="Mindestmenge_Hamburg")

In [ ]:
len(model.getConss())

## Lösung

In [ ]:
model.redirectOutput()

In [ ]:
model.optimize()

In [ ]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', model.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if model.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(model.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if model.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(model.getVal(Y[i,a])))

In [ ]:
model.getStatus()